In [1]:
import pandas as pd
import numpy as np

In [2]:
#pip install google-adk
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

In [3]:
import APIKey    #Storing my API key in python class "APIKey"
import os

google_API = APIKey.GOOGLESTUDIO_API_KEY    #My google studio API

os.environ["GOOGLE_API_KEY"] = google_API

In [4]:
import yfinance as yf

ticker = "SHEL"
shell = yf.Ticker(ticker)

shell_stock_price = yf.download(ticker, period = "5y", auto_adjust=False)
shell_stock_price

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SHEL,SHEL,SHEL,SHEL,SHEL,SHEL
Date,,,,,,
2020-11-19,27.532658,33.180000,33.224998,32.480000,32.660000,4333768
2020-11-20,27.930962,33.660000,33.709999,33.165001,33.380001,4869829
2020-11-23,29.217140,35.209999,35.220001,34.619999,34.669998,6213612
2020-11-24,30.793758,37.110001,37.224998,36.744999,36.849998,7285970
2020-11-25,30.154818,36.340000,36.599998,35.810001,36.340000,5255506
...,...,...,...,...,...,...
2025-11-12,75.689377,76.410004,76.785004,76.370003,76.370003,3912880
2025-11-13,75.203995,75.919998,76.820000,75.665001,76.440002,4360533


# Info

In [5]:
info = shell.info
info

{'address1': 'Shell Centre',
 'city': 'London',
 'zip': 'SE1 7NA',
 'country': 'United Kingdom',
 'phone': '44 20 7934 1234',
 'website': 'https://www.shell.com',
 'industry': 'Oil & Gas Integrated',
 'industryKey': 'oil-gas-integrated',
 'industryDisp': 'Oil & Gas Integrated',
 'sector': 'Energy',
 'sectorKey': 'energy',
 'sectorDisp': 'Energy',
 'longBusinessSummary': 'Shell plc operates as an energy and petrochemical company Europe, Asia, Oceania, Africa, the United States, and other Americas. It operates through Integrated Gas; Upstream; Marketing; Chemicals and Products; and Renewables and Energy Solutions segments. The company explores for and extracts natural gas to produce liquefied natural gas or convert into gas-to-liquids products; explores for and extracts crude oil and natural gas liquids; and operates upstream and midstream infrastructure to deliver gas to market. It is also involved in marketing supplies fuels and lubricants for transport, manufacturing, mining, power ge

In [6]:
# Using summary as part of the Ai prompt
summary = info['longBusinessSummary']
summary

'Shell plc operates as an energy and petrochemical company Europe, Asia, Oceania, Africa, the United States, and other Americas. It operates through Integrated Gas; Upstream; Marketing; Chemicals and Products; and Renewables and Energy Solutions segments. The company explores for and extracts natural gas to produce liquefied natural gas or convert into gas-to-liquids products; explores for and extracts crude oil and natural gas liquids; and operates upstream and midstream infrastructure to deliver gas to market. It is also involved in marketing supplies fuels and lubricants for transport, manufacturing, mining, power generation, agriculture, and construction industries; operates electric vehicle charging and convenience retail; turn crude oil and other feedstocks into products for households, industry, and transport; trades crude oil, oil products, and petrochemicals; and oil sand activities. In addition, the company generates, markets, and trades power from wind, solar and pipeline ga

In [7]:
# Comparing some financial data with its competitors 
compare_df = pd.DataFrame(columns = ['Ticker', 'Dividend Yield', 'Trailing PE', 'TTM PS', 'Profit Margin', 'PB Ratio', 'Trailing EPS', 'EV/EBITDA', 
                                     'Current Ratio', 'Debt-to-Equity', 'ROA', 'ROE', 'PEG Ratio'])

shell_list = [ticker, info['dividendYield'], info['trailingPE'], info['priceToSalesTrailing12Months'], info['profitMargins'], info['priceToBook'], 
             info['trailingEps'], info['enterpriseToEbitda'], info['currentRatio'], info['debtToEquity'], info['returnOnAssets'],
             info['returnOnEquity'], info['trailingPegRatio']] 

compare_df.loc[0] = shell_list
compare_df

,Ticker,Dividend Yield,Trailing PE,TTM PS,Profit Margin,PB Ratio,Trailing EPS,EV/EBITDA,Current Ratio,Debt-to-Equity,ROA,ROE,PEG Ratio
0,SHEL,3.8,15.369387,0.802272,0.05438,2.489175,4.9,9.868,1.352,41.602,0.04901,0.08155,1.634


In [8]:
# Find the companies' main competitors
industry = yf.Industry(info['industryKey'])

competitors = list(industry.top_companies.index.values)
competitors

['XOM', 'CVX', 'NFG', 'UNTC']

In [9]:
# Adding the competitors' financial data to make comparison
for comp in competitors:
    comp_info = yf.Ticker(comp).info
    comp_list = [comp, comp_info['dividendYield'], comp_info['trailingPE'], comp_info['priceToSalesTrailing12Months'], comp_info['profitMargins'],
                 comp_info['priceToBook'], comp_info['trailingEps'], comp_info['enterpriseToEbitda'], comp_info['currentRatio'], 
                 comp_info['debtToEquity'], comp_info['returnOnAssets'], comp_info['returnOnEquity'], comp_info['trailingPegRatio']]
    compare_df.loc[len(compare_df)] = comp_list

compare_df

C:\Users\User\AppData\Local\Temp\ipykernel_15808\820696664.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = comp_list
C:\Users\User\AppData\Local\Temp\ipykernel_15808\820696664.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = comp_list


,Ticker,Dividend Yield,Trailing PE,TTM PS,Profit Margin,PB Ratio,Trailing EPS,EV/EBITDA,Current Ratio,Debt-to-Equity,ROA,ROE,PEG Ratio
0,SHEL,3.80,15.369387,0.802272,0.05438,2.489175,4.90,9.868,1.352,41.602,0.04901,0.08155,1.6340
1,XOM,3.50,17.300870,1.555439,0.09181,1.926488,6.88,8.626,1.137,15.672,0.05108,0.11424,2.7756
2,CVX,4.42,21.606188,1.640373,0.06767,1.629333,7.11,9.323,1.152,21.239,0.04047,0.07316,2.9249
3,NFG,2.66,14.318663,3.227398,0.22766,2.375292,5.68,7.132,0.444,91.548,0.07008,0.17449,NaN
4,UNTC,15.08,4.930267,1.328413,0.23337,1.355939,6.74,2.907,3.545,1.009,0.14997,0.26941,NaN


In [10]:
# Round the number for a more effective AI agent
compare_df = compare_df.round(4)

# Convert the dataframe to string for AI prompt
compare_df_string = compare_df.to_markdown(index=False)
compare_df_string

'| Ticker   |   Dividend Yield |   Trailing PE |   TTM PS |   Profit Margin |   PB Ratio |   Trailing EPS |   EV/EBITDA |   Current Ratio |   Debt-to-Equity |    ROA |    ROE |   PEG Ratio |\n|:---------|-----------------:|--------------:|---------:|----------------:|-----------:|---------------:|------------:|----------------:|-----------------:|-------:|-------:|------------:|\n| SHEL     |             3.8  |       15.3694 |   0.8023 |          0.0544 |     2.4892 |           4.9  |       9.868 |           1.352 |           41.602 | 0.049  | 0.0815 |      1.634  |\n| XOM      |             3.5  |       17.3009 |   1.5554 |          0.0918 |     1.9265 |           6.88 |       8.626 |           1.137 |           15.672 | 0.0511 | 0.1142 |      2.7756 |\n| CVX      |             4.42 |       21.6062 |   1.6404 |          0.0677 |     1.6293 |           7.11 |       9.323 |           1.152 |           21.239 | 0.0405 | 0.0732 |      2.9249 |\n| NFG      |             2.66 |       14.318

# News AI Agent 
### With news generate by google search

In [11]:
google_news_agent = Agent(
    name="GoogleNewsAgent",
    model="gemini-2.5-flash",
    instruction="""You are a specialized news arrange agent for a given company. Your job is to read the company's summary and use Google Search 
    to find news relate to the company and generate comment with some important points that might affect the company's stock price.""",
    tools = [google_search],
    output_key="google_news_arrangement",
)

In [12]:
runner = InMemoryRunner(agent=google_news_agent)
prompt = f"""{ticker} summary: 

{summary}"""
response = await runner.run_debug(prompt)


 ### Created new session: debug_session_id

User > SHEL summary: 

Shell plc operates as an energy and petrochemical company Europe, Asia, Oceania, Africa, the United States, and other Americas. It operates through Integrated Gas; Upstream; Marketing; Chemicals and Products; and Renewables and Energy Solutions segments. The company explores for and extracts natural gas to produce liquefied natural gas or convert into gas-to-liquids products; explores for and extracts crude oil and natural gas liquids; and operates upstream and midstream infrastructure to deliver gas to market. It is also involved in marketing supplies fuels and lubricants for transport, manufacturing, mining, power generation, agriculture, and construction industries; operates electric vehicle charging and convenience retail; turn crude oil and other feedstocks into products for households, industry, and transport; trades crude oil, oil products, and petrochemicals; and oil sand activities. In addition, the company ge

# Comparing AI Agent

In [13]:
competitors_agent = Agent(
    name="CompetitorsAgent",
    model="gemini-2.5-flash",
    instruction="""You are a specialized agent to compare the target company to its competitors. Your job is to analyse the financial ratio for 
    the given company and its competitors. Finally summary the most important points and provide some insights that might affect the target 
    companies' stock price.""",
    output_key="comparing_competitors",
)

In [14]:
runner = InMemoryRunner(agent=competitors_agent)
prompt = f""" Please analyze the following financial data for SHEL and its competitors:

{compare_df_string}"""
response = await runner.run_debug(prompt)


 ### Created new session: debug_session_id

User >  Please analyze the following financial data for SHEL and its competitors:

| Ticker   |   Dividend Yield |   Trailing PE |   TTM PS |   Profit Margin |   PB Ratio |   Trailing EPS |   EV/EBITDA |   Current Ratio |   Debt-to-Equity |    ROA |    ROE |   PEG Ratio |
|:---------|-----------------:|--------------:|---------:|----------------:|-----------:|---------------:|------------:|----------------:|-----------------:|-------:|-------:|------------:|
| SHEL     |             3.8  |       15.3694 |   0.8023 |          0.0544 |     2.4892 |           4.9  |       9.868 |           1.352 |           41.602 | 0.049  | 0.0815 |      1.634  |
| XOM      |             3.5  |       17.3009 |   1.5554 |          0.0918 |     1.9265 |           6.88 |       8.626 |           1.137 |           15.672 | 0.0511 | 0.1142 |      2.7756 |
| CVX      |             4.42 |       21.6062 |   1.6404 |          0.0677 |     1.6293 |           7.11 |      